In [2]:
import database
from snowballing.operations import load_work, reload, work_by_varname
from snowballing.selenium_scholar import SeleniumScholarQuerier


querier = SeleniumScholarQuerier()
querier.apply_settings(10, 4)

from snowballing.snowballing import ArticleNavigator


[ INFO]  settings applied


In [3]:
class FindNames(ArticleNavigator):
    """Widget for curating database"""

    def __init__(self, querier, names, force=False, debug=False, index=0):
        reload()
        self.names = names
        self.force = force
        self.querier = querier
        self.next_page_widget = Button(description='Next Work', icon='fa-arrow-right')
        self.reload_widget = Button(description='Reload', icon='fa-refresh')
        self.previous_page_widget = Button(description='Previous Work', icon='fa-arrow-left')
        self.debug_widget = ToggleButton(value=debug, description="Debug")
        self.textarea_widget = ToggleButton(value=False, description="TextArea")
        self.page_number_widget = Label(value="")
        self.output_widget = Output()
        self.next_page_widget.on_click(self.next_page)
        self.reload_widget.on_click(self.reload)
        self.previous_page_widget.on_click(self.previous_page)
        self.textarea_widget.observe(self.show)
        self.view = VBox([
            HBox([
                self.previous_page_widget,
                self.reload_widget,
                self.next_page_widget,
                self.debug_widget,
                self.textarea_widget,
                self.page_number_widget
            ]),
            self.output_widget
        ])
        self.index = index
        self.name = ""
        self.articles = []
        self.reload(show=False)


    def next_page(self, b):
        """Go to next page"""
        self.index = min(len(self.names) - 1, self.index + 1)
        self.reload(b)

    def previous_page(self, b):
        """Go to previous page"""
        self.query = max(0, self.index - 1)
        self.reload(b)

    def set_index(self):
        """Set page index"""
        self.page_number_widget.value = str(self.index)
        self.next_page_widget.disabled = self.index == len(self.names) - 1
        self.previous_page_widget.disabled = self.index == 0

    def show(self, b=None):
        """Show comparison"""
        self.output_widget.clear_output()
        with self.output_widget:
            print(self.name)
            if not self.articles:
                print("<unknown>")
                return
            try:
                article, _, _ = self.articles[0]
                if 'div' in article:
                    self.show_article_html(article['div'])
                else:
                    print(article['name'])
                for article, nwork, info in self.valid_articles([article], show=True):
                    self.show_article(article, nwork, info)
            except:
                traceback.print_exc(file=sys.stdout)
                print(self.varname, '<error>')

    def reload(self, b=None, show=True):
        """Reload"""
        self.output_widget.clear_output()
        with self.output_widget:
            if self.debug_widget.value:
                ScholarConf.LOG_LEVEL = 3
            else:
                ScholarConf.LOG_LEVEL = 2
            reload()
            self.querier.tasks.clear()

            if self.index >= len(self.names):
                self.set_index()
                return
            self.name = self.names[self.index]
            print(self.name)
            from .selenium_scholar import SearchScholarQuery
            query = SearchScholarQuery()

            query.set_scope(False)
            query.set_words(self.name)
            query.set_num_page_results(1)
            self.querier.send_query(query)

            self.articles = self.querier.articles
        if show:
            self.show()

        self.set_index()

    def browser(self):
        """Present widget"""
        self.show()
        return self.view

    def _ipython_display_(self):
        """ Displays widget """
        self.show()
        display(self.view)


In [7]:
import pandas as pd
df = pd.read_excel("todo_francisco.xlsx")

In [8]:
df

,0,1,2
0,A case study comparing defect profiles of a re...,0.5 - garbett1999a - 0\n0.4939759036144578 - l...,springer2015
1,A case study on software risk analysis and pla...,0.5524861878453039 - dag2002a - 0\n0.539473684...,springer2015
2,A critical look at ISO 9000 for software quali...,0.6355140186915887 - bubevski2016a - 0\n0.6126...,springer2015
3,A flexible method for software effort estimati...,0.5945945945945946 - wilkie2007a - 0\n0.573913...,springer2015
4,A Guttman Scaling of CMM Level 2 Practices: In...,0.5326633165829145 - armstrong2016a - 0\n0.526...,springer2015
5,A lightweight approach to software validation ...,0.4648648648648649 - bubevski2014a - 0\n0.4615...,springer2015
6,A quality cost reduction model for large-scale...,0.72 - deshpande2012a - 0\n0.6542056074766355 ...,springer2015
7,A quality driven extension to the QVT-relation...,0.5333333333333333 - prahalad1999a - 0\n0.5079...,springer2015
8,A research framework for building SPI proposal...,0.5604395604395604 - pino2009b - 0\n0.51428571...,springer2015
9,A socio-technical approach to improving the sy...,0.631578947368421 - taxen2005a - 0\n0.57746478...,springer2015
